In [1]:
date = 20220930
#year = 2022 #current year
#year1 = 202207 #current month

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


Branch_Dsc = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\02_User_Defined\01_Branch_Dsc\Branch_Dsc.txt", sep = "\t", header = 0, low_memory = False)
Branch_Dsc.columns = Branch_Dsc.columns.str.replace(" ", "_")
Branch_Dsc.Branch_Cd = Branch_Dsc.Branch_Cd.astype(str)

TOTAL_IMPAIRED_FINAL  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(date)[:4]+'\\'+str(date)[:6]+'\working\Total_Impaired_Final_'+str(date)+'.txt',sep = ",", header = 0, low_memory = False)
#TOTAL_IMPAIRED_FINAL  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(year)+'\\'+str(year1)+'\working\Total_Impaired_Final_'+str(date)+'.txt',sep = ",", header = 0, low_memory = False)
TOTAL_IMPAIRED_FINAL.columns = TOTAL_IMPAIRED_FINAL.columns.str.replace(" ", "_")
TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER = TOTAL_IMPAIRED_FINAL.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)
TOTAL_IMPAIRED_FINAL.columns = TOTAL_IMPAIRED_FINAL.columns.str.replace("(", "_").str.replace(")", "_").str.replace(".", "_").str.replace("-", "_").str.replace("\n", "_").str.replace(" ", "_").str.replace(":", "_")


TOTAL_IMPAIRED_FINAL_1 = TOTAL_IMPAIRED_FINAL[['V_ORIGINAL_ACCOUNT_NUMBER',\
'CUSTOMER_ID',\
'V_D_CUST_FIRST_NAME',\
'OUTSTANDING_AFTER_EIR_UWI',\
'PRODUCT_GROUP',\
'MKT_SUB_SEGMENT',\
'V_PROD_GROUP_DESC',\
'PROD_LV1',\
'PROD_LV5',\
'PROD_LV6',\
'F_RPSIA_FLAG',\
'F_NCIF_CUST',\
'V_STAGE_REASON_DESC',\
'V_D_CUST_INDUSTRY',\
'V_PROD_CODE',\
'V_PROD_DESC',\
'V_PROD_FAMILY_DESC',\
'V_ACCT_BRANCH_CODE']]



conn = sqlite3.connect(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\07_RSME\\'+str(date)+'\SMEBB_IF.db')
c = conn.cursor()

c.execute("CREATE TABLE IF NOT EXISTS SMEBBIF_TOTAL_IMPAIRED_FINAL({})".format(",".join(TOTAL_IMPAIRED_FINAL_1.columns)))

for row in TOTAL_IMPAIRED_FINAL_1.iterrows():
    sql = "INSERT INTO SMEBBIF_TOTAL_IMPAIRED_FINAL({}) VALUES ({})".format(','.join(TOTAL_IMPAIRED_FINAL_1.columns), ','.join(['?']*len(TOTAL_IMPAIRED_FINAL_1.columns)))
    c.execute(sql, tuple(row[1]))

conn.commit()


c.execute("CREATE TABLE IF NOT EXISTS SMEBBIF_branch ({})".format(','.join(Branch_Dsc.columns)))

for row in Branch_Dsc.iterrows():
    sql = "INSERT INTO SMEBBIF_branch({}) VALUES ({})".format(','.join(Branch_Dsc.columns), ','.join(['?']*len(Branch_Dsc.columns)))
    c.execute(sql, tuple(row[1]))

conn.commit()

c.execute("""
CREATE TABLE LIST_BB AS SELECT
A.V_ORIGINAL_ACCOUNT_NUMBER AS ACCT_NO,
A.CUSTOMER_ID AS CUS_NO,
A.V_D_CUST_FIRST_NAME AS NAME,
A.OUTSTANDING_AFTER_EIR_UWI AS OUTSTANDING_AFTER_EIR_UWI,
A.PRODUCT_GROUP,
A.MKT_SUB_SEGMENT,
A.V_PROD_GROUP_DESC AS PROD_GROUP,
A.PROD_LV1,
A.PROD_LV5,
A.PROD_LV6,
A.F_RPSIA_FLAG,
A.F_NCIF_CUST,
V_STAGE_REASON_DESC,
A.V_D_CUST_INDUSTRY AS NOB,
/*A.V_INDUSTRY_CATEGORY_DESC AS NOB_DESC,*/
A.V_PROD_CODE AS PROD_CODE,
A.V_PROD_DESC AS PROD_DESC,
A.V_PROD_FAMILY_DESC AS PROD_FAMILY_DESC,
A.V_STAGE_REASON_DESC AS REASON,
A.V_ACCT_BRANCH_CODE AS ACCT_BRANCH_CODE,
B.BRANCH_CD,
B.BRANCH,
B.DESCRIPTION,
B.REGION_CD,
B.REGION_DESC,
B.STATE_CD,
B.STATE_DESC,
B.ZONE_CD,
B.ZONE_DESC,
B.BUS_CTR_CD,
B.BUS_CTR_DESC,
B.BRANCH_DESC,
B.BASE_BRANCH_DESC

FROM SMEBBIF_TOTAL_IMPAIRED_FINAL A

LEFT JOIN SMEBBIF_branch B
ON A.V_ACCT_BRANCH_CODE = B.BRANCH_CD
/*ON A.V_ACCT_BRANCH_CODE = COMPRESS(PUT(B.BRANCH_CD, 4.))*/
/*PUT(A.V_ACCT_BRANCH_CODE, $4.)*/

WHERE A.PRODUCT_GROUP = '04.BB' AND A.PROD_LV1 = 'On Balance Sheet'
ORDER BY 5,4; """)

conn.commit()

c.execute("""
CREATE TABLE LIST_SME AS SELECT
A.V_ORIGINAL_ACCOUNT_NUMBER AS ACCT_NO,
A.CUSTOMER_ID AS CUS_NO,
A.V_D_CUST_FIRST_NAME AS NAME,
A.OUTSTANDING_AFTER_EIR_UWI AS OUTSTANDING_AFTER_EIR_UWI,
A.PRODUCT_GROUP,
A.MKT_SUB_SEGMENT,
A.V_PROD_GROUP_DESC AS PROD_GROUP,
A.PROD_LV1,
A.PROD_LV5,
A.PROD_LV6,
A.F_RPSIA_FLAG,
A.F_NCIF_CUST,
V_STAGE_REASON_DESC,
A.V_D_CUST_INDUSTRY AS NOB,
/*A.V_INDUSTRY_CATEGORY_DESC AS NOB_DESC,*/
A.V_PROD_CODE AS PROD_CODE,
A.V_PROD_DESC AS PROD_DESC,
A.V_PROD_FAMILY_DESC AS PROD_FAMILY_DESC,
A.V_STAGE_REASON_DESC AS REASON,
A.V_ACCT_BRANCH_CODE AS ACCT_BRANCH_CODE,
B.BRANCH_CD,
B.BRANCH,
B.DESCRIPTION,
B.REGION_CD,
B.REGION_DESC,
B.STATE_CD,
B.STATE_DESC,
B.ZONE_CD,
B.ZONE_DESC,
B.BUS_CTR_CD,
B.BUS_CTR_DESC,
B.BRANCH_DESC,
B.BASE_BRANCH_DESC

FROM SMEBBIF_TOTAL_IMPAIRED_FINAL A

LEFT JOIN SMEBBIF_branch B
ON A.V_ACCT_BRANCH_CODE = B.BRANCH_CD

WHERE A.PRODUCT_GROUP = '05.SME' AND A.PROD_LV1 = 'On Balance Sheet'
ORDER BY 5,4;
""")

conn.commit()

c.execute("""
CREATE TABLE SUMM_SME_NOCROSSSTAGING AS SELECT
COUNT(A.V_ORIGINAL_ACCOUNT_NUMBER) AS ACCT_NO,
COUNT(CUSTOMER_ID) AS CUS_NO,
SUM(A.OUTSTANDING_AFTER_EIR_UWI) AS OUTSTANDING_AFTER_EIR_UWI,
A.PRODUCT_GROUP,
A.F_NCIF_CUST

FROM SMEBBIF_TOTAL_IMPAIRED_FINAL A

LEFT JOIN SMEBBIF_branch B
ON A.V_ACCT_BRANCH_CODE = B.BRANCH_CD

WHERE A.PRODUCT_GROUP = '05.SME' AND A.PROD_LV1 = 'On Balance Sheet' AND V_STAGE_REASON_DESC NOT IN ('Bankruptcy Flag Y',
'Customer Override Due to Impairment Status Y',
'Customer Override Due to NPL Flag Y',
'Customer Override Due to Rescheduled'
)
GROUP BY 4,5;
""")

conn.commit()



#SINI
writer2 = pd.ExcelWriter(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\07_RSME\\'+str(date)+'\\IF_SMEBB_'+str(date)[:6]+'(python).xlsx', engine='xlsxwriter')

LIST_BB = pd.read_sql_query("SELECT * FROM LIST_BB", conn)
LIST_BB.to_excel(writer2, sheet_name='IF_BB', index = False)

LIST_SME = pd.read_sql_query("SELECT * FROM LIST_SME", conn)
LIST_SME.to_excel(writer2, sheet_name='IF_SME', index = False)

SUMM_SME_NOCROSSSTAGING = pd.read_sql_query("SELECT * FROM SUMM_SME_NOCROSSSTAGING", conn)
SUMM_SME_NOCROSSSTAGING.to_excel(writer2, sheet_name='SUMM_SME_NO_CROSSSTAGING', index = False)

writer2.save()


# DB SECTION:
SQLite in Python (Outsource)

In [6]:
conn.close()

In [ ]:
c.execute("drop table SUMM_003")
conn.commit() 

In [ ]:
cek = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name ;", conn)
cek